<a href="https://colab.research.google.com/github/victorma969/Treinamento-IEEE/blob/main/WineQuality-Periodo-3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importação Bibliotecas & Extração dos dados



In [ ]:
#importa-se a base de dados
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from time import time
 
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/3 período /winequality.csv")

df.head()




,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,wine_is_red
0,0,5.9,0.61,0.08,2.1,0.071,16.0,24.0,0.99376,3.56,0.77,11.1,6,1
1,1,8.2,1.00,0.09,2.3,0.065,7.0,37.0,0.99685,3.32,0.55,9.0,6,1
2,2,6.4,0.24,0.49,5.8,0.053,25.0,120.0,0.99420,3.01,0.98,10.5,6,0
3,3,6.6,0.23,0.26,1.3,0.045,16.0,128.0,0.99340,3.36,0.60,10.0,6,0
4,4,7.2,0.24,0.29,3.0,0.036,17.0,117.0,0.99411,3.36,0.68,10.1,6,0


Processamento dos subsets

In [ ]:
import plotly.express as px

fig = px.scatter_3d(df, x = 'wine_is_red' ,y ='alcohol' ,z ='residual sugar', color = 'quality')

fig.show()

Implementação K-NN 

Processamento dos dados

In [ ]:
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score



#reorganiza o data set
label = df['wine_is_red'].replace([False,True],[0,1])
y1 = df['wine_is_red'].replace([False,True],[0,1])
y2 = df['quality'].replace([False,True],[0,1])

#Remove a feature 'wine_is_red' porque é do tipo label label
df.drop(['wine_is_red','quality'],axis=1,inplace=True) 

df.reset_index(drop=True, inplace=True)


In [ ]:
#separando os conjuntos de treino e teste (75% e 25%)
train_df, test_df, train_label, test_label = train_test_split(df,label,test_size = 0.25,random_state = 33)
train_df, test_df, train_label, test_label = np.array(train_df), np.array(test_df), np.array(train_label), np.array(test_label)

Distância Euclidiana e *Split* da base de dados.


In [ ]:
# Função para cálculo do KNN
def KNN(x,y,k):
    dist = []
    #calcula a distância euclidiana 
    dist_euclidiana = np.sqrt(np.sum((x-y)**2, axis=1))
    #concatena a distância euclidiana com a label do dado
    label_arr = np.column_stack((train_label,dist_euclidiana))
    #ordena pelas distâncias euclidianas 
    label_dist_set = label_arr[label_arr[:,1].argsort()]
    #Conta os K vizinhos
    count = Counter(label_dist_set[0:k,0])
    #Retorna a label baseado na contagem dos valores de dados associados 
    keys, value = list(count.keys()), list(count.values())
    if len(value)>1:
        if value[0]>value[1]:
            return int(keys[0])
        else:
            return int(keys[1])
    else:
        return int(keys[0])
             

Métricas de avaliação do classificador K-NN

Aplicação no conjunto de treino:

In [ ]:
#aplicação do K-nn no conjunto de treino
pred = []
for i in range(len(train_df)):
    pred.append(KNN(train_df,train_df[i],5))

print(classification_report(pred, train_label))

              precision    recall  f1-score   support

           0       0.97      0.94      0.96      3794
           1       0.81      0.91      0.86      1078

    accuracy                           0.93      4872
   macro avg       0.89      0.92      0.91      4872
weighted avg       0.94      0.93      0.93      4872



In [ ]:
init = time()
#aplicação do K-nn no conjunto de test
test_pred = []
for i in range(len(test_df)):
    test_pred.append(KNN(train_df,test_df[i],7))
end = time()

print(label)
print(end-init)
print(classification_report(test_pred, test_label))

0       1
1       1
2       0
3       0
4       0
       ..
6492    0
6493    0
6494    0
6495    0
6496    0
Name: wine_is_red, Length: 6497, dtype: int64
1.3064377307891846
              precision    recall  f1-score   support

           0       0.97      0.93      0.95      1293
           1       0.76      0.88      0.81       332

    accuracy                           0.92      1625
   macro avg       0.86      0.90      0.88      1625
weighted avg       0.92      0.92      0.92      1625



Normalização dos dados e efeito no tempo de execução do algoritimo.

In [ ]:
from sklearn.preprocessing import StandardScaler
#import matplotlib.pyplot as plt


scaler = StandardScaler()  
scaler.fit(df)
df_norm = scaler.transform(df)


#separando os conjuntos de treino e teste (75% e 25%)
train_df_norm, test_df_norm, train_label_norm, test_label_norm = train_test_split(df_norm,label,test_size = 0.25,random_state = 33)
train_df_norm, test_df_norm, train_label_norm, test_label_norm = np.array(train_df_norm), np.array(test_df_norm), np.array(train_label_norm), np.array(test_label_norm)

In [ ]:
init = time()
#aplicação do K-nn no conjunto de test normalizado
test_pred = []
for i in range(len(test_df_norm)):
    test_pred.append(KNN(train_df_norm,test_df_norm[i],5))
end = time()

print(end-init)

1.9980876445770264


Implementando o RandomForest para classificação binária e multiclasse

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

x = df


#separa o dataset em treino e testa para a label y1
x1_forest_train,x1_forest_test, y1_train, y1_test = train_test_split(x,y1,random_state = 33, test_size = 0.25)

#separa o dataset em treino e testa para a label y2
x2_forest_train,x2_forest_test, y2_train, y2_test = train_test_split(x,y2,random_state = 33, test_size = 0.25)








0.9944615384615385


Classificação Binária "wine_is_red"

In [ ]:
#classificação binária
model_random_forest = RandomForestClassifier(n_estimators = 100, n_jobs= -1, random_state = 50)

model_random_forest.fit(x1_forest_train,y1_train)

y1_pred = model_random_forest.predict(x1_forest_test)

print("Acuracia do modelo igual à:",accuracy_score(y1_test,y1_pred)*100,"%")

Acuracia do modelo igual à: 99.32307692307693 %


Classificação Multiclasse  "quality"


In [ ]:
model_random_forest.fit(x2_forest_train,y2_train)

y2_pred = model_random_forest.predict(x2_forest_test)

print("Acuracia do modelo igual à:",accuracy_score(y2_test,y2_pred)*100,"%")
print(y2_pred)

Acuracia do modelo igual à: 68.43076923076923 %
[6 7 6 ... 7 6 6]
